In [19]:
import pandas as pd

pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 30)

teams = pd.DataFrame({
    "teamID" : [1,2,3,4],
    "teamName" : ["Real Madrid","Man City","Bayern","AC Milan"],
    "country" : ["Spain","England","Germany","Italy"]
})
print(teams)

matches = pd.DataFrame({
    "matchID" : [101,102,103,104],
    "homeID" : [1,2,3,4],
    "awayID" : [2,3,4,1],
    "homeGoals" : [1,3,2,0],
    "awayGoals" : [1,2,1,2],
    "stage" : ["Quarter","Quarter","Semi","Semi"]
})
print(matches)


df = matches.merge(teams, left_on="homeID", right_on="teamID", how="left")
df = df.rename(columns={"teamName" : "homeTeam" , "country" : "homeCountry"})

df = df.merge(teams, left_on="awayID", right_on="teamID", how="left")
df = df.rename(columns={"teamName" : "awayTeam" , "country" : "awayCountry"})

print(df[["matchID","homeTeam","awayTeam","homeGoals","awayGoals","stage"]])

df["result"] = df.apply(lambda x:
                        "homeWin" if x.homeGoals > x.awayGoals else
                        "awayWin" if x.homeGoals < x.awayGoals else "draw"
                        , axis=1)

df["totalGoals"] = df["homeGoals"] + df["awayGoals"]

print(df[["result","totalGoals"]])

pivot = pd.pivot_table(
    pd.concat([
        df[["stage","homeTeam","homeGoals"]].rename(columns={"homeTeam":"team", "homeGoals" : "goals"}),
        df[["stage","awayTeam","awayGoals"]].rename(columns={"awayTeam":"team", "awayGoals" : "goals"})
    ]),
    values="goals",
    index="stage",
    columns="team",
    aggfunc="sum",
    fill_value=0
)

print(pivot)

melted = pivot.reset_index().melt(id_vars="stage", var_name="team",value_name="goals")
melted = melted.sort_values(["stage","goals"], ascending=[True,False])
print(melted)

melted["goals"] = pd.to_numeric(melted["goals"], errors="coerce")
melted = melted.sort_values(["stage","goals","team"], ascending=[True,False,True])

top2 = melted.groupby("stage", group_keys=False).head(2)

print(top2)

top1 = (
    melted.groupby("team")["goals"].sum()
    .reset_index(name="total_goals")
    .sort_values("total_goals", ascending=False)
    .head(1)
)
print(top1)

mvp = melted.groupby("stage", group_keys=False).head(1)

print(mvp)

melted["stage_total"] = (
    melted.groupby("stage")["goals"].transform("sum")
)

# 3) Safe percentage calculation: each team's share of goals in its stage
eps = 1e-12
melted["pct_in_stage"] = melted["goals"] / (melted["stage_total"] + eps)

# Optional: show as percent string for display (keep numeric column too if you like)
melted["pct_in_stage_pct"] = (melted["pct_in_stage"] * 100).round(2)

# 4) Rank within each stage by contribution (1 = biggest share)
#    method="dense" gives ranks like 1,1,2,2,3 (ties get the same rank)
melted["rank_in_stage"] = (
    melted.groupby("stage")["pct_in_stage"].rank(method="dense", ascending=False)
)

# 5) Build a clean, sorted output table
out = (melted[["stage","team","goals","stage_total","pct_in_stage","pct_in_stage_pct","rank_in_stage"]]
       .sort_values(["stage","pct_in_stage","team"], ascending=[True, False, True])
       .reset_index(drop=True))

print(out)

# 6) Sanity check: for each stage, shares should sum to ~1.0
print("\nShare sums per stage (should be ~1.0):")
print(melted.groupby("stage")["pct_in_stage"].sum().round(6))


   teamID     teamName  country
0       1  Real Madrid    Spain
1       2     Man City  England
2       3       Bayern  Germany
3       4     AC Milan    Italy
   matchID  homeID  awayID  homeGoals  awayGoals    stage
0      101       1       2          1          1  Quarter
1      102       2       3          3          2  Quarter
2      103       3       4          2          1     Semi
3      104       4       1          0          2     Semi
   matchID     homeTeam     awayTeam  homeGoals  awayGoals    stage
0      101  Real Madrid     Man City          1          1  Quarter
1      102     Man City       Bayern          3          2  Quarter
2      103       Bayern     AC Milan          2          1     Semi
3      104     AC Milan  Real Madrid          0          2     Semi
    result  totalGoals
0     draw           2
1  homeWin           5
2  homeWin           3
3  awayWin           2
team     AC Milan  Bayern  Man City  Real Madrid
stage                                         